In [ ]:
!pip install langchain_openai

In [ ]:
from openai import OpenAI
from getpass import getpass

In [ ]:
print("Insertar el API Key de OpenAI")
openai_key = getpass()

# Modelos

Instanciando el modelo

In [ ]:
# Importamos la clase ChatOpenAI desde LangChain,
# que nos permite conectarnos a modelos de OpenAI como GPT-4, GPT-3.5, etc.
from langchain_openai import ChatOpenAI

# Creamos una instancia del modelo de lenguaje (LLM).
llm = ChatOpenAI(
    # Especificamos el modelo a usar. En este caso "gpt-4o-mini".
    model="gpt-4o-mini",
    
    # La temperatura controla la "creatividad" o "aleatoriedad" de las respuestas:
    # - Valores bajos (ej. 0.0 o 0.2): respuestas más predecibles, consistentes y deterministas.
    # - Valores medios (ej. 0.7): balance entre creatividad y coherencia.
    # - Valores altos (ej. 1.0 o más): respuestas más variadas, creativas, pero menos controladas.
    temperature=0.7,
    
    openai_api_key=openai_key
)


## Prueba de mensaje simple

In [ ]:
respuesta = llm.invoke("Hola")
print(respuesta)

## Analisis de la respuesta

In [ ]:
import json

data = respuesta.__dict__
print(json.dumps(data, indent=2, ensure_ascii=False))

In [ ]:
#dejamos solo el string de la respuesta que nos interesa sin el json
print(respuesta.content)

## Lista de mensajes

In [ ]:
messages = [
    (
        "system",
        "Eres un asistente que traduce del español a binario. Traduce las oraciones del usuario",
    ),
    ("human", "Me gusta la programacion"),
]
ai_msg = llm.invoke(messages)
ai_msg
data = ai_msg.__dict__
print(json.dumps(data, indent=2, ensure_ascii=False))

## Chaining

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente que traduce del {input_language} al {output_language}. Traduce las oraciones del usuario",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
respuesta = chain.invoke(
    {
        "input_language": "español",
        "output_language": "binario",
        "input": "Me gusta la programacion",
    }
)
respuesta

In [ ]:
print(respuesta.content)

# Messages

Los mensajes son la unidad de comunicación en los [modelos de chat](/docs/concepts/chat_models). Se utilizan para representar la entrada y salida de un modelo de chat, así como cualquier contexto adicional o metadatos que puedan estar asociados a una conversación.

Cada mensaje tiene un **rol** (por ejemplo, "user", "assistant") y un **contenido** (por ejemplo, texto o datos multimodales), junto con metadatos adicionales que varían según el proveedor del modelo de chat.

LangChain proporciona un formato de mensaje unificado que puede usarse en diferentes modelos de chat, lo que permite a los usuarios trabajar con distintos modelos sin preocuparse por los detalles específicos del formato de mensajes de cada proveedor.

## ¿Qué contiene un mensaje?

Un mensaje normalmente consta de la siguiente información:

* **Rol**: El rol del mensaje (por ejemplo, "user", "assistant").
* **Contenido**: El contenido del mensaje (por ejemplo, texto o datos multimodales).
* Metadatos adicionales: id, nombre, [uso de tokens](/docs/concepts/tokens) y otros metadatos específicos del modelo.

### Rol

Los roles se utilizan para distinguir entre diferentes tipos de mensajes en una conversación y ayudar al modelo de chat a entender cómo responder a una secuencia dada de mensajes.

| **Rol**               | **Descripción**                                                                                                                                                                                                                                              |
| --------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **system**            | Se utiliza para indicar al modelo de chat cómo comportarse y proporcionar contexto adicional. No es compatible con todos los proveedores de modelos.                                                                                                         |
| **user**              | Representa la entrada de un usuario que interactúa con el modelo, generalmente en forma de texto u otra entrada interactiva.                                                                                                                                 |
| **assistant**         | Representa una respuesta del modelo, que puede incluir texto o una solicitud para invocar herramientas.                                                                                                                                                      |
| **tool**              | Mensaje utilizado para devolver al modelo los resultados de la invocación de una herramienta, después de haber obtenido datos o realizado un procesamiento externo. Usado con modelos que admiten [invocación de herramientas](/docs/concepts/tool_calling). |
| **function** (legado) | Rol heredado, correspondiente a la API antigua de llamadas a funciones de OpenAI. Debería usarse el rol **tool** en su lugar.                                                                                                                                |

### Contenido

El contenido de un mensaje puede ser texto o una lista de diccionarios que representan [datos multimodales](/docs/concepts/multimodality) (por ejemplo, imágenes, audio, video). El formato exacto del contenido puede variar entre diferentes proveedores de modelos de chat.

Actualmente, la mayoría de modelos de chat admiten texto como tipo principal de contenido, aunque algunos también admiten datos multimodales. Sin embargo, el soporte para datos multimodales aún es limitado.

## Estructura de la conversación

La secuencia de mensajes en un modelo de chat debe seguir una estructura específica para garantizar que el modelo genere una respuesta válida.

Por ejemplo, una estructura típica de conversación podría ser:

1. **Mensaje del usuario**: "Hola, ¿cómo estás?"
2. **Mensaje del asistente**: "Estoy bien, gracias por preguntar."
3. **Mensaje del usuario**: "¿Puedes contarme un chiste?"
4. **Mensaje del asistente**: "¡Claro! ¿Por qué el espantapájaros ganó un premio? Porque era sobresaliente en su campo."

## Mensajes de LangChain

LangChain proporciona un formato unificado de mensajes que puede usarse con todos los modelos de chat, lo que permite a los usuarios trabajar con distintos modelos sin preocuparse por los detalles del formato de mensajes de cada uno.

Los mensajes de LangChain son objetos de Python que heredan de [BaseMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.base.BaseMessage.html).

Los cinco tipos principales de mensajes son:

* [SystemMessage](#systemmessage): corresponde al rol **system**
* [HumanMessage](#humanmessage): corresponde al rol **user**
* [AIMessage](#aimessage): corresponde al rol **assistant**
* [AIMessageChunk](#aimessagechunk): también corresponde al rol **assistant**, pero se usa para respuestas por [streaming](/docs/concepts/streaming)
* [ToolMessage](#toolmessage): corresponde al rol **tool**

In [ ]:
# Importamos los tipos de mensajes que se usan en LangChain para simular un historial de conversación
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Definimos una lista de mensajes que representan el historial de la conversación
mensajes = [
    # Mensaje del sistema: define el rol y comportamiento del asistente
    SystemMessage(content='Eres un asistente que resuelve operaciones matemáticas y cuya respuesta es directa.'),
    # Primer turno de conversación: el humano pregunta
    HumanMessage(content='¿Cuánto es 1 + 1?'),
    # El asistente responde
    AIMessage(content='2'),
    # Segundo turno: otra pregunta del humano
    HumanMessage(content='¿Cuánto es 10 * 5?'),
    # Respuesta del asistente
    AIMessage(content='50'),
    # Tercer turno: otra pregunta
    HumanMessage(content='¿Cuánto es 10 + 3?'),
]

# Ejemplo de cómo invocar al modelo directamente con la lista de mensajes (comentado)
# chat.invoke(mensajes)
# Aquí se hace streaming de la respuesta del modelo usando los mensajes anteriores como contexto.
# El modelo generará la respuesta en fragmentos ("chunks").
for chunk in llm2 .stream(mensajes):
    # Imprime cada pedazo de la respuesta generada en tiempo real
    print(chunk.content)


Esto es similar a la estructura de mensajes de la API de OpenAI, pero con una sintaxis diferente:

```python
mensajes = [
    {'role': 'user', 'content': 'Cuánto es 1 + 1'},
    {'role': 'assistant', 'content': '2'},
    {'role': 'user', 'content': 'Cuánto es 10 * 5'},
    {'role': 'assistant', 'content': '50'},
    {'role': 'user', 'content': 'Cuánto es 10 + 3'},
]
```

In [ ]:
## Probando otros modelos

In [ ]:
!pip install langchain-nvidia-ai-endpoints

In [ ]:

#obtener api key de https://build.nvidia.com/

from getpass import getpass
print("Insertar el API Key de Nvidia")
nvidia_key = getpass()

In [ ]:
## Core LC Chat Interface
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm2 = ChatNVIDIA(model="meta/llama-4-maverick-17b-128e-instruct", api_key=nvidia_key)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

mensajes = [
    SystemMessage(content='Eres un asistente que resuelve operaciones matemáticas y cuya respuesta es directa.'),
    HumanMessage(content='¿Cuánto es 1 + 1?'),
    AIMessage(content='2'),
    HumanMessage(content='¿Cuánto es 10 * 5?'),
    AIMessage(content='50'),
    HumanMessage(content='¿Cuánto es 10 + 3?'),
]

# chat.invoke(mensajes)

for chunk in llm2.stream(mensajes):
    print(chunk.content)

# Streams

La generación de respuestas completas desde LLM suele conllevar un retraso de varios segundos, que se hace más evidente en aplicaciones complejas con múltiples llamadas a modelos. Afortunadamente, los LLM generan respuestas de forma iterativa, lo que permite mostrar resultados intermedios a medida que se generan. Al transmitir estos resultados intermedios, LangChain facilita una experiencia de usuario más fluida en aplicaciones basadas en LLM y ofrece compatibilidad integrada con la transmisión en su diseño.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

mensajes = [
    SystemMessage(content='Eres un asistente que cuenta chistes.'),
    HumanMessage(content='¿Cuánto es 1 + 1?')
]

for tramo in llm.stream(mensajes):
    print(tramo.content, end='')

# Caching

In [ ]:
!pip install langchain-community

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

mensajes = [
    SystemMessage(content='Eres un narrador de historias cortas'),
    HumanMessage(content='Sobre programacion')
]

In [ ]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

set_llm_cache(InMemoryCache())

Primera vez

In [ ]:
%%time
llm2.invoke(mensajes)

Segunda vez

In [ ]:
%%time
llm2.invoke(mensajes)

In [ ]:
%%time
llm2.invoke(mensajes)

# Output Parsers

In [ ]:
review_cliente = """Estos audífonos inalámbricos superaron completamente mis expectativas. Tienen
cancelación de ruido activa, modo transparencia y duración de batería de hasta 30 horas. Los pedí
el lunes y llegaron el miércoles, perfecto para mi viaje de trabajo del viernes. El sonido es
cristalino y los graves son impresionantes. Mi hermano me había recomendado esta marca después de
probárselos. Aunque costaron $180 dólares, que es más de lo que normalmente gastaría en audífonos,
definitivamente valen cada centavo por la calidad de construcción y las características premium que
ofrecen. Los he usado durante largas sesiones de trabajo y son muy cómodos."""

from langchain.prompts import ChatPromptTemplate

review_template = ChatPromptTemplate.from_template("""
Para el siguiente texto, extrae la siguiente información:

recomendacion_externa: ¿El producto fue recomendado por alguien más? True si es verdadero y False si es falso
o no se encuentra la información.

dias_entrega: ¿Cuántos días tardó en llegar la entrega? Si no se encuentra la respuesta, devuelve -1.

precio_mencionado: Extrae cualquier precio específico mencionado en el review. Si no hay precio específico,
devuelve null.

caracteristicas_destacadas: Extrae las características técnicas o funcionales mencionadas del producto.
Devuelve el resultado como una lista en Python.

justificacion_precio: Extrae cualquier frase que justifique el costo o valor del producto. Devuelve el
resultado como una lista en Python.

Texto: {review}

Retorna la respuesta en formato JSON
""")

print(review_template.format_messages(review=review_cliente))

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

respuesta = llm.invoke(review_template.format_messages(review=review_cliente))
print(respuesta.content)